In [ ]:
!pip install sentence-transformers faiss-cpu transformers


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
from transformers import pipeline


In [ ]:
corpus = [
    "Machine Learning is a field of AI that allows systems to learn from data.",
    "Supervised learning uses labeled data.",
    "Unsupervised learning finds patterns without labels.",
    "Deep learning uses neural networks with many layers."
]


In [ ]:
# Load embedding model
embed_model = SentenceTransformer('all-MiniLM-L6-v2')
corpus_embeddings = embed_model.encode(corpus)

# Build FAISS index
dimension = corpus_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(corpus_embeddings)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
generator = pipeline("text-generation", model="distilgpt2", max_new_tokens=100)


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
chat_history = []

def ask_question(question):
    global chat_history

    # Embed question
    q_embed = embed_model.encode([question])

    # Search FAISS
    _, I = index.search(q_embed, k=2)  # top 2 relevant docs
    context = " ".join([corpus[i] for i in I[0]])

    # Add previous chat history
    memory = " ".join(chat_history)

    # Prepare prompt
    prompt = f"Context: {context}\nChat History: {memory}\nQuestion: {question}\nAnswer:"

    # Generate answer
    answer = generator(prompt)[0]['generated_text']

    # Save to history
    chat_history.append(f"Q: {question}")
    chat_history.append(f"A: {answer}")

    return answer


In [ ]:
# 1️⃣ Imports
from sentence_transformers import SentenceTransformer
import faiss
from transformers import pipeline
import numpy as np

# 2️⃣ Corpus
corpus = [
    "Machine Learning is a field of AI that allows systems to learn from data.",
    "Supervised learning uses labeled data.",
    "Unsupervised learning finds patterns without labels.",
    "Deep learning uses neural networks with many layers."
]

# 3️⃣ Embeddings
embed_model = SentenceTransformer('all-MiniLM-L6-v2')
corpus_embeddings = embed_model.encode(corpus, convert_to_numpy=True, normalize_embeddings=True).astype('float32')

# 4️⃣ FAISS index
dimension = corpus_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(corpus_embeddings)

# 5️⃣ LLM pipeline
generator = pipeline("text-generation", model="distilgpt2", max_new_tokens=100, pad_token_id=50256)

# 6️⃣ Chatbot memory
chat_history = []

def ask_question(question, k=2):
    global chat_history

    # Embed question
    q_embed = embed_model.encode([question], convert_to_numpy=True, normalize_embeddings=True).astype('float32')

    # Search FAISS
    distances, I = index.search(q_embed, k=min(k, len(corpus)))  # Prevent asking more than corpus
    context = " ".join([corpus[i] for i in I[0] if i < len(corpus)])

    # Chat history
    memory = " ".join(chat_history)

    # Prompt for generation
    prompt = f"Context: {context}\nChat History: {memory}\nQuestion: {question}\nAnswer:"

    # Generate answer
    answer = generator(prompt)[0]['generated_text']

    # Save to history
    chat_history.append(f"Q: {question}")
    chat_history.append(f"A: {answer}")

    return answer


Device set to use cpu


In [ ]:
print(ask_question("What is machine learning?"))
print(ask_question("Explain deep learning."))


Context: Machine Learning is a field of AI that allows systems to learn from data. Supervised learning uses labeled data.
Chat History: 
Question: What is machine learning?
Answer: In machine learning, there are many types of computers that may be used for machine learning. For example, machine learning may be a system that learns from data, which is what the human brain learns from experience.
You can learn about the human brain for yourself in the conversation below.
Question: What about the natural language?
Answer: In machine learning, there are many types of computers that may be used for machine learning. For example, machine learning may be a system that learns from experience
Context: Deep learning uses neural networks with many layers. Machine Learning is a field of AI that allows systems to learn from data.
Chat History: Q: What is machine learning? A: Context: Machine Learning is a field of AI that allows systems to learn from data. Supervised learning uses labeled data.
Cha